In [1]:
#원본 데이터 txt 파일 불러오기
import pandas as pd
import numpy as np
df = pd.read_csv("서울시 자치동별 독거노인 현황.txt", sep = "\t", header = 1)

In [2]:
df.columns

Index(['기간', '자치구', '동', '계', '65세~79세', '80세이상', '계.1', '65세~79세.1',
       '80세이상.1', '계.2', '65세~79세.2', '80세이상.2', '계.3', '65세~79세.3', '80세이상.3',
       '비고'],
      dtype='object')

In [3]:
#컬럼 이름 재지정
df.columns = ['기간', '자치구', '동', '총합계', '65~79세', '80세이상', '기초생활수급권자합계', '65세~79세.1',
       '80세이상.1', '저소득노인합계','65세~79세.2', '80세이상.2', '일반합계', '65세~79세.3',
       '80세이상.3', '비고']

In [4]:
#불필요한 column drop
df = df.drop(['65~79세', '80세이상', '65세~79세.1',
       '80세이상.1', '65세~79세.2', '80세이상.2', '65세~79세.3',
       '80세이상.3', '비고' ,'기초생활수급권자합계', '저소득노인합계', '일반합계'], axis = 1)

In [5]:
#불필요한 row 제거
df = df[df['동'] != '소계']
df = df[df['동'] != '합계']

In [6]:
#결측치 -에서 0으로 바꾸기(문자면 계산 시 오류)
df = df.replace(to_replace ='-', value = '0', regex = True)

In [7]:
#geo-json에 맞춰서 동 이름 변환
df = df.replace(to_replace ='종로5·6가동', value = '종로5.6가동', regex = True)

In [8]:
#결측된 둔촌 1동을 둔촌2동 데이터로 보정
df['총합계'].loc[(df['동'] == '둔촌1동')] = 741

C:\Users\bkdon\Anaconda3\lib\site-packages\pandas\core\indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


In [9]:
#규칙에 맞게 column 이름 재지정
df.columns = ['기간', '구', '동', '행정동별독거노인합계']

In [10]:
#행정동코드 merge를 위한 xlsx 파일 로드
data = pd.read_excel("geojson_행정동코드.xlsx")

In [11]:
#행정동코드 merge
df= pd.merge(df, data, on=["구", "동"] ,  how='left')

In [12]:
#column 순서 재배치
df = df[['행정동코드', '구', '동', '행정동별독거노인합계']]

In [13]:
#전처리한 데이터를 xlsx 로 저장
df.to_excel('독거노인.xlsx', encoding = 'utf-8')